## Import necessary libraries: 

In [ ]:
import tensorflow as tf
import keras as K 
tf.__version__, K.__version__


In [ ]:
# load processing func
import source.toolkit as tk
import source.dojo_tools as dj 

from source.toolkit import list_channels
from source.toolkit import list_channels_df

# load main class object for monitoring blood cells
from source.SickleML_Monitor import CountAdheredBloodCells 

# Loading tools for extracting gdrive data 
import source.load_data_tools as loading_tools

import os
import time
import pandas as pd

import sys
import gdown 

## Download pretrained weights from gdrive: 

Important to note, if you haven't downloaded the pretrained weights (e.g Resnet50 and encoder-decoder model) in your working directory, you must run the below code. Don't worry if it takes several minutes to load because there is several network weights to download. In addition, by default the remaining pretrained models found in the supplementary material is suppressed with quotations. You are also more then welcome the depress the comment quotations and download those weights onto your working directory. Overall, all of the pretrained weights will be saved in automatic folder called "./Phase2_Pretrained-models/*" or "./Phase1_Pretrained-models/*"

In [ ]:

' Resnet50 Xception pretrained weights '

os.makedirs(f'./Phase2_Pretrained-models/Resnet50/', exist_ok = True)
os.chdir(f'./Phase2_Pretrained-models/Resnet50/') 

loading_tools.resnet50_gdrive_()
    
os.chdir(f'../')
os.chdir(f'../')



' Download encoder-decoder weights '

os.makedirs(f'./Phase1_Pretrained-models/ce-jaccard_encoder-decoder-net/', exist_ok = True)
os.chdir(f'./Phase1_Pretrained-models/ce-jaccard_encoder-decoder-net/') 

loading_tools.ce_jaccard_gdrive_()
    
os.chdir(f'../')
os.chdir(f'../')


"""
Below are functions to load the other pretrained models found in the Supplementary materials ....
"""

"""

' Vanilla Xception pretrained weights '

os.makedirs(f'./Phase2_Pretrained-models/Vanilla/', exist_ok = True)
os.chdir(f'./Phase2_Pretrained-models/Vanilla/') 

loading_tools.Vanilla_gdrive_()

os.chdir(f'../')
os.chdir(f'../')



' Download hr-net weights '

os.makedirs(f'./Phase1_Pretrained-models/hrnet_ce-jaccard_hr-net/', exist_ok = True)
os.chdir(f'./Phase1_Pretrained-models/hrnet_ce-jaccard_hr-net/') 

loading_tools.hrnet_CeJaccard_gdrive_()

os.chdir(f'../')
os.chdir(f'../')




' Download Xception pretrained weights '

os.makedirs(f'./Phase2_Pretrained-models/Xception/', exist_ok = True)
os.chdir(f'./Phase2_Pretrained-models/Xception/') 

loading_tools.Xception_gdrive_()

os.chdir(f'../')
os.chdir(f'../')

"""

## Download test whole channel images from gdrive:
Here you will just download two test channel images into your working directory. Sorry, we cannot share more channels that was used for this study because the remaining channels are patient confidential. Overall, all of the downloaded test channels will be located in the new directory folder called "./data/Manual-VS-AI/"

In [ ]:
' Download channel images from gdrive ... '

channel_path= './data/Manual-VS-AI/'
os.makedirs(channel_path, exist_ok=True)
os.chdir(channel_path)
loading_tools.laminin_channel_gdrive_()
os.chdir('../')
os.chdir('../')
channel_filenames  =  os.listdir(channel_path)


In [ ]:
print(f'Number of channels: {len(channel_filenames)}')
channel_filenames

#### Additional functions

In [ ]:
'  Here, we have a function that automatically loads the whole ensemble model (5 neural networks) '

def load_ensembles(Phase1_path_model, Phase2_path_model):

    Phase1_path = './Phase1_Pretrained-models/' + Phase1_path_model + '/'# folder for Phase I 
    Phase1_filenames = os.listdir(Phase1_path)
    Phase1_final_filenames = []
    for ii in range(len(Phase1_filenames)):
        if ii % 2 != 0:
            Phase1_final_filenames.append(Phase1_filenames[ii].replace('.json', ''))

    Phase2_path = './Phase2_Pretrained-models/' + Phase2_path_model + '/'# folder for Phase II
    Phase2_filenames = os.listdir(Phase2_path)
    Phase2_final_filenames = []
    for ii in range(len(Phase2_filenames)):
        if ii % 2 != 0:
            Phase2_final_filenames.append(Phase2_filenames[ii].replace('.json',''))


    Phase1_ensemble = tk.load_zoo(Phase1_path, Phase1_final_filenames) # loading the Phase I ensemble (expect: 7)
    Phase2_ensemble = tk.load_zoo(Phase2_path, Phase2_final_filenames) # loading the Phase I ensemble (expect: 5)
    return Phase1_ensemble, Phase2_ensemble

' function for creating dataframes while computing cell counts during inference '

def create_final_df(counts, times):
    counts_df = pd.DataFrame(counts)
    counts_df.columns = ['filename', 'def-sRBC', 'nondef-sRBC', 'Other']
    times_df = pd.DataFrame(times)
    times_df.columns = ['time_secs']
    final_df = pd.concat([counts_df, times_df], axis = 1)
    return final_df

## Inference: end-to-end predictions 

Here, we will input a whole channel image which will be segmented so that individual adhered cells can be extracted for sRBC morphology classification.

In [ ]:
%%time 




counts, times = [], [] 
count_container, time_container = [], []

rbc_thres = [0.4]
wbc_thres = [0.4]
other_thres = [0.9]

Phase1_names, Phase2_names = 'ce-jaccard_encoder-decoder-net', 'Resnet50'

Phase1_ensemble, Phase2_ensemble = load_ensembles(Phase1_names, Phase2_names)
counts, times = [], [] 

for index, filenames in enumerate(channel_filenames):
    for rep in ((".png", ""), (".jpg", "")):
        clean_filename = filenames.replace(*rep)
    print('Analysis:', index, '| Channel:', clean_filename)
    print('==================================================================')
    channel = CountAdheredBloodCells(channel_path, filenames) # calling the class object
    # calling the function to output cell counts
    start = time.time()
    sRBC, WBC, Others, img_container, sRBC_container, WBC_container, Other_container = channel.call_pipeline(Phase1_ensemble, Phase2_ensemble, rbc_thres, wbc_thres, other_thres)
    end = time.time()
    run_time = end-start
            
    times.append([run_time])
    counts.append([filenames, sRBC, WBC, Others])
            
    final_df = create_final_df(counts,times)
   # final_df.to_csv(f'./AI-vs-Human_counts/{Phase1_name}_{Phase2_name}.csv', index = False)
            


In [ ]:
print("Here is the predicted counts ...")
final_df